In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from rtb.datasets.fake import FakeProductDataset

In [23]:
ds = FakeProductDataset(root="../../data", process=True)

processing db...
processing db took 0.00 seconds.
reindexing pkeys and fkeys...
reindexing pkeys and fkeys took 0.00 seconds.
saving table product...
done in 0.00 seconds.
saving table customer...
done in 0.00 seconds.
saving table review...
done in 0.00 seconds.


In [34]:
ds

FakeProductDataset(
  tables=['customer', 'product', 'review'],
  tasks=['churn', 'ltv'],
  min_time=1970-01-01 00:00:00,
  max_time=1983-08-31 00:00:00,
  train_max_time=1980-12-06 00:00:00,
  val_max_time=1982-04-19 00:00:00,
)

In [24]:
db = ds._db

In [25]:
db.tables["product"]

Table(df=
    product_id       category    title     price
0            0           None  title_1  6.523037
1            1             []  title_2  1.541314
2            2  [toy, health]  title_3  0.894173
3            3           None  title_1  4.216620
4            4             []  title_2  1.061467
5            5  [toy, health]  title_3  3.684594
6            6           None  title_1  9.772576
7            7             []  title_2  1.702511
8            8  [toy, health]  title_3  8.169753
9            9           None  title_1  8.844051
10          10             []  title_2  2.860061
11          11  [toy, health]  title_3  0.007624
12          12           None  title_1  3.153043
13          13             []  title_2  4.093500
14          14  [toy, health]  title_3  9.904809
15          15           None  title_1  5.728990
16          16             []  title_2  5.695456
17          17  [toy, health]  title_3  5.493068
18          18           None  title_1  8.238289
19        

In [26]:
db.tables["review"]

Table(df=
     customer_id  product_id review_time  rating
0             74          17  1970-01-01       4
1             59           2  1970-01-11       1
2             79          29  1970-01-21       3
3             10          24  1970-01-31       4
4             95          24  1970-02-10       1
..           ...         ...         ...     ...
495           47          15  1983-07-22       4
496            8          29  1983-08-01       3
497           36           4  1983-08-11       2
498            0          23  1983-08-21       3
499           21           9  1983-08-31       4

[500 rows x 4 columns],
  fkey_col_to_pkey_table={'customer_id': 'customer', 'product_id': 'product'},
  pkey_col=None,
  time_col=review_time)

In [27]:
db.tables["customer"]

Table(df=
    customer_id  age  gender
0             0   43    male
1             1   30  female
2             2   34    male
3             3   41  female
4             4   35    male
..          ...  ...     ...
95           95   45  female
96           96   43    male
97           97   35  female
98           98   18    male
99           99   19  female

[100 rows x 3 columns],
  fkey_col_to_pkey_table={},
  pkey_col=customer_id,
  time_col=None)

## LTV

In [28]:
window_size = ds.tasks["ltv"].window_sizes[0]
window_size

Timedelta('364 days 00:00:00')

In [29]:
train_table = ds.make_train_table("ltv", window_size)
train_table

Table(df=
        window_min_time     window_max_time  customer_id       ltv  count
0   1970-01-01 00:00:01 1970-12-31 00:00:01            0  0.000000      0
1   1970-01-01 00:00:01 1970-12-31 00:00:01            1  0.000000      0
2   1970-01-01 00:00:01 1970-12-31 00:00:01            2  0.000000      0
3   1970-01-01 00:00:01 1970-12-31 00:00:01            3  7.729392      2
4   1970-01-01 00:00:01 1970-12-31 00:00:01            5  0.000000      0
..                  ...                 ...          ...       ...    ...
995 1978-12-21 00:00:01 1979-12-20 00:00:01           42  0.000000      0
996 1978-12-21 00:00:01 1979-12-20 00:00:01           81  2.860061      1
997 1978-12-21 00:00:01 1979-12-20 00:00:01           97  0.000000      0
998 1970-01-01 00:00:01 1970-12-31 00:00:01           11  0.000000      0
999 1978-12-21 00:00:01 1979-12-20 00:00:01           61  0.894173      1

[1000 rows x 5 columns],
  fkey_col_to_pkey_table={'customer_id': 'customer'},
  pkey_col=None,
  tim

In [30]:
test_table = ds.make_test_table("ltv", window_size)
test_table

Table(df=
       window_min_time window_max_time  customer_id
0  1982-04-19 00:00:01      1983-04-18            0
1  1982-04-19 00:00:01      1983-04-18            1
2  1982-04-19 00:00:01      1983-04-18            2
3  1982-04-19 00:00:01      1983-04-18            3
4  1982-04-19 00:00:01      1983-04-18            4
..                 ...             ...          ...
95 1982-04-19 00:00:01      1983-04-18           98
96 1982-04-19 00:00:01      1983-04-18           99
97 1982-04-19 00:00:01      1983-04-18           53
98 1982-04-19 00:00:01      1983-04-18           59
99 1982-04-19 00:00:01      1983-04-18           79

[100 rows x 3 columns],
  fkey_col_to_pkey_table={'customer_id': 'customer'},
  pkey_col=None,
  time_col=window_min_time)

## Churn

In [31]:
window_size = ds.tasks["churn"].window_sizes[0]
window_size

Timedelta('364 days 00:00:00')

In [32]:
train_table = ds.make_train_table("churn", window_size)
train_table

Table(df=
        window_min_time     window_max_time  customer_id  churn
0   1970-01-01 00:00:01 1970-12-31 00:00:01            0   True
1   1970-01-01 00:00:01 1970-12-31 00:00:01            1   True
2   1970-01-01 00:00:01 1970-12-31 00:00:01            2   True
3   1970-01-01 00:00:01 1970-12-31 00:00:01            3  False
4   1970-01-01 00:00:01 1970-12-31 00:00:01            4   True
..                  ...                 ...          ...    ...
995 1978-12-21 00:00:01 1979-12-20 00:00:01           95  False
996 1978-12-21 00:00:01 1979-12-20 00:00:01           96  False
997 1978-12-21 00:00:01 1979-12-20 00:00:01           97   True
998 1978-12-21 00:00:01 1979-12-20 00:00:01           98   True
999 1978-12-21 00:00:01 1979-12-20 00:00:01           99   True

[1000 rows x 4 columns],
  fkey_col_to_pkey_table={'customer_id': 'customer'},
  pkey_col=None,
  time_col=window_min_time)

In [33]:
test_table = ds.make_test_table("churn", window_size)
test_table

Table(df=
       window_min_time window_max_time  customer_id
0  1982-04-19 00:00:01      1983-04-18            0
1  1982-04-19 00:00:01      1983-04-18            1
2  1982-04-19 00:00:01      1983-04-18            2
3  1982-04-19 00:00:01      1983-04-18            3
4  1982-04-19 00:00:01      1983-04-18            4
..                 ...             ...          ...
95 1982-04-19 00:00:01      1983-04-18           95
96 1982-04-19 00:00:01      1983-04-18           96
97 1982-04-19 00:00:01      1983-04-18           97
98 1982-04-19 00:00:01      1983-04-18           98
99 1982-04-19 00:00:01      1983-04-18           99

[100 rows x 3 columns],
  fkey_col_to_pkey_table={'customer_id': 'customer'},
  pkey_col=None,
  time_col=window_min_time)